# Análise de Embeddings de Poemas de Clarice Lispector e Medida de Criatividade

Este notebook coleta poemas curtos de Clarice Lispector via OpenAI API, gera embeddings usando diferentes modelos e reduz dimensões para visualizar a similaridade entre os poemas. Além disso, propõe uma medida de criatividade baseada na variabilidade semântica entre as linhas de cada poema.

## 1. Configuração e importações

In [1]:

import os
from dotenv import load_dotenv
load_dotenv()
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
from main import EmbeddingVisualizer
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.metrics.pairwise import cosine_distances


/Users/joaomadruga/.pyenv/versions/default/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Carregamento de poemas do arquivo JSON

In [2]:
import json
with open('clarice_lispector_poemas.json', encoding='utf-8') as f:
    data = json.load(f)
poemas = {item['name']: item['poem'] for item in data['poems']}
print(f"Carregados {len(poemas)} poemas.")


Carregados 16 poemas.


## 3. Geração de embeddings e redução de dimensionalidade

In [3]:

# Instancia o visualizador de embeddings
visualizer = EmbeddingVisualizer()
model_name = "MiniLM-L6"
# Embeddings dos poemas completos
titulos = list(poemas.keys())
textos = list(poemas.values())
embeddings = visualizer.compute_embeddings(textos, model_name)
# Redução para 2D com PCA
coords = visualizer.reduce_dimensions(embeddings, method="pca", n_components=2)
df = pd.DataFrame(coords, columns=["x", "y"])
df["Título"] = titulos
fig = px.scatter(df, x="x", y="y", text="Título", title="PCA dos poemas de Clarice Lispector")
fig.update_traces(textposition="top center")
fig.show()


/Users/joaomadruga/Desktop/projects/embedding-visualizer/main.py:36: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  OpenAIEmbeddings(
Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-base-en and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias

## 4. Medida de criatividade baseada na variabilidade semântica

In [4]:

scores = {}
for titulo, texto in poemas.items():
    linhas = [l for l in texto.splitlines() if l.strip()]
    if len(linhas) < 2:
        scores[titulo] = 0.0
        continue
    emb_linhas = visualizer.compute_embeddings(linhas, model_name)
    dists = cosine_distances(emb_linhas)
    n = len(linhas)
    valores = dists[np.triu_indices(n, 1)]
    scores[titulo] = float(np.mean(valores))
scores_df = pd.DataFrame({"Título": list(scores.keys()), "Criatividade": list(scores.values())})
scores_df = scores_df.sort_values("Criatividade", ascending=False)
fig2 = px.bar(scores_df, x="Título", y="Criatividade", title="Medida de criatividade dos poemas")
fig2.show()


## 5. Conclusões (em Português)
Observamos que a redução de dimensionalidade mostrou agrupamentos de poemas semanticamente próximos. A medida de criatividade, definida como a média das distâncias semânticas entre as linhas de cada poema, indica que poemas com maior variabilidade interna são potencialmente mais criativos. No entanto, essa métrica serve apenas como um indicador inicial e deve ser refinada com análises qualitativas adicionais e validação com especialistas em literatura.